In [1]:
from bs4 import BeautifulSoup
import requests
import csv

In [2]:
# xsseed archive URL
url = "http://www.xssed.com/archive/page={}/"

In [3]:
mirrors = []
# we go through every page of the archive and collect all the links 
# for websites vulnerable to XSS attacks
for i in range(1,1530):
    # get html and javascript content
    r = requests.get(url.format(i))
    # we use beautiful soup library for web scraping
    soup = BeautifulSoup(r.content,"html5lib")
    # using soup we find all 'a' tags
    # 'a' tags are
    for link in soup.find_all('a'):
        if "mirror" in link.get("href"):
            mirrors.append(link.get("href"))

ConnectionError: HTTPConnectionPool(host='www.xssed.com', port=80): Max retries exceeded with url: /archive/page=534/ (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x00000173BF003680>: Failed to resolve 'www.xssed.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
url = "http://www.xssed.com{}"
malicious_urls = []
mirror_urls = []

for m in mirrors:
    mirror = url.format(m)
    r = requests.get(mirror)
    soup = BeautifulSoup(r.content,"html5lib")
    malicious_urls.append(soup.select("#contentpaneOpen > table > tbody > tr:nth-child(4) > th")[0].text[5:])
    mirror_urls.append(soup.select("#contentpaneOpen > table > tbody > tr:nth-child(5) > th > a")[0].get("href"))

In [ ]:
def write_file(filename,strings):
    with open(filename, "w") as file:
        for string in strings:
            file.write(string + "\n")

write_file("mirrors.txt",mirror_urls)
write_file("malicious_urls.txt",malicious_urls)

In [ ]:
def file_to_list(filename):
    lines = []
    with open(filename,'r') as file:
        for line in file:
            cleaned_line = line.strip()
            lines.append(cleaned_line)
    return lines

In [ ]:
len(file_to_list())

In [ ]:
import csv
import random

data = []
def generate_csv():
    malicious = file_to_list("malicious_urls.txt")
    benign = file_to_list("benign_urls.txt")
    for m in malicious:
        data.append([m,1])
    for b in benign:
        data.append([b,0])
    random.shuffle(data)

    with open("urls.csv",mode="w",newline="") as file:
        csv_writer = csv.writer(file)
        for row in data:
            print(row)
            csv_writer.writerow(row)
    file.close()

In [ ]:
generate_csv()

In [ ]:
def read_csv(filename):
    with open(filename,mode="r") as file:
        csv_reader = csv.reader(file)
        data = []
        for row in csv_reader:
            data.append(row)
        return data

In [ ]:
import re
import urllib

def duplicate(url):
    patterns = [r'<<', r'>>', r'%3C%3']
    total_occurrences = 0
    for pattern in patterns:
        occurrences = len(re.findall(pattern, url))
        total_occurrences += occurrences
    return total_occurrences

def special(url):
    substrings = ["“, “”>", "“>", "“/>"]
    total_occurrences = 0
    for substring in substrings:
        total_occurrences += url.count(substring)
    return total_occurrences

def html_tags(url):
    pattern = r'<[^>]+>'
    html_tags = re.findall(pattern, url)
    return len(html_tags)

def cookie(url):
    pattern = r'document\.cookie'
    if re.search(pattern, url):
        return 1
    return 0

def redirections(url):
    pattern = r'window\.(location\.href|history\.back|navigate)'
    if re.search(pattern, url):
       return 1
    return 0

def extract_features(url):
    # attackers use url encoding to hide payloads / attack vectors
    # hence we decode it
    decoded_url = urllib.parse.unquote(url)

    return [len(decoded_url), 1 if decoded_url == url else 0 ,html_tags(decoded_url),duplicate(decoded_url),special(decoded_url),cookie(decoded_url),redirections(decoded_url)]

In [ ]:
dataset = [["number_of_characters","url_encoding","html_tags","duplicate_characters","cookie_requests","redirections","target"]]

with open("urls.csv",mode="r") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        new_row = extract_features(row[0])
        new_row.append(int(row[1]))
        dataset.append(new_row)
file.close()

with open("urls_features_data.csv",mode="w",newline="") as file:
    csv_writer = csv.writer(file)
    for row in dataset:
        csv_writer.writerow(row)

NameError: name 'csv' is not defined

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_csv("urls_features_data.csv")
X = df.drop('target',axis=1)
y = df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9890582624146638
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7069
           1       1.00      0.97      0.98      3624

    accuracy                           0.99     10693
   macro avg       0.99      0.98      0.99     10693
weighted avg       0.99      0.99      0.99     10693



In [ ]:
from sklearn.svm import SVC

X_train, X_test, y_tain, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC(kernel='linear', C=1)  
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9890582624146638
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7069
           1       1.00      0.97      0.98      3624

    accuracy                           0.99     10693
   macro avg       0.99      0.98      0.99     10693
weighted avg       0.99      0.99      0.99     10693



In [ ]:
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_tain, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = DecisionTreeClassifier(random_state=42)  
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9890582624146638
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7069
           1       1.00      0.97      0.98      3624

    accuracy                           0.99     10693
   macro avg       0.99      0.98      0.99     10693
weighted avg       0.99      0.99      0.99     10693

